In [1]:
import pandas as pd

from trainer import Trainer
from split_data_type import SplitDataType
from feature_transformer import TreeSimilarFeatureTransformar
from estimator import ForestEstimator

In [2]:
target_column = "SalePrice"

train_df = pd.read_csv("../HousePrice_train.csv")
test_X = pd.read_csv("../HousePrice_test.csv")
test_y = pd.read_csv("../sample_submission.csv")[target_column]

In [3]:
tree_features = TreeSimilarFeatureTransformar(train_df, SplitDataType.TRAIN, logging=True)

In [4]:
tree_features.drop_not_needed()

----------------------------------------------------------------------------------------------------
TRAIN: drop_not_needed
Size = (1460, 80)
Nulls columns = 19
----------------------------------------------------------------------------------------------------


In [5]:
tree_features.fill_null()

----------------------------------------------------------------------------------------------------
TRAIN: fill_null
Size = (1460, 80)
Nulls columns = 0
----------------------------------------------------------------------------------------------------


In [6]:
forest_estimator = ForestEstimator()

In [7]:
train_df = train_df.select_dtypes(exclude=['object'])

In [8]:
forest_estimator.fit(train_df.drop(columns=[target_column]), train_df[target_column])

0.9993150684931507
